<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Define-constants" data-toc-modified-id="Define-constants-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Define constants</a></span><ul class="toc-item"><li><span><a href="#Load-NLP-feats" data-toc-modified-id="Load-NLP-feats-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Load NLP feats</a></span></li><li><span><a href="#Load-volume-bars-and-time-bars" data-toc-modified-id="Load-volume-bars-and-time-bars-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Load volume bars and time bars</a></span></li><li><span><a href="#Build-LUT" data-toc-modified-id="Build-LUT-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Build LUT</a></span></li></ul></li></ul></div>

In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
from cqcplot import *

/home/cqc5/anaconda3/envs/tensorflow_cpu/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/cqc5/anaconda3/envs/tensorflow_cpu/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/cqc5/anaconda3/envs/tensorflow_cpu/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/cqc5/anaconda3/envs/tensorflow_cpu/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
from arrow_utils import *
from arrow_clouds.data_scalers import *

Using TensorFlow backend.


# Define constants

In [46]:
RES_UNIT = 'S'
RES = 5
VOLUME = 50

## Load NLP feats

In [47]:
path = '../processed_data/features/nlp_reports/nlp_features.pkl'
reports_df = pd.read_pickle(path)

In [48]:
reports_df.head()

,Pre_Doc2Vec_f1,Pre_Doc2Vec_f2,Pre_Doc2Vec_f3,Pre_Doc2Vec_f4,Pre_Doc2Vec_f5,Pre_Doc2Vec_f6,Pre_Doc2Vec_f7,Pre_Doc2Vec_f8,Pre_Doc2Vec_f9,Pre_Doc2Vec_f10,...,gram_f291,gram_f292,gram_f293,gram_f294,gram_f295,gram_f296,gram_f297,gram_f298,gram_f299,gram_f300
date,,,,,,,,,,,,,,,,,,,,,
2012-12-24 08:02:00,-0.344248,-0.347838,-0.175678,-0.079595,-0.048744,-0.038706,-0.267172,-1.008872,-0.057370,-0.325998,...,0.025654,0.0,0.0,0.05119,0.051743,0.064384,0.02474,0.051237,0.076085,0.033200
2013-01-01 06:35:00,-0.006598,0.023259,-0.133878,-0.011773,-0.061570,0.041536,-0.286036,0.081177,-0.181855,-0.016568,...,0.000000,0.0,0.0,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
2013-01-04 18:42:00,-0.002963,-0.015582,-0.104835,0.143139,0.077515,0.030537,-0.191341,0.255992,-0.029815,-0.026122,...,0.000000,0.0,0.0,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
2013-01-07 07:39:00,0.118311,-0.069228,-0.264404,0.142889,0.032075,0.149364,-0.095372,0.072506,0.043688,0.096990,...,0.000000,0.0,0.0,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
2013-01-07 08:21:00,0.060745,0.077263,-0.517718,-0.257046,0.057317,0.245623,-0.398536,-0.235504,0.191876,0.096845,...,0.000000,0.0,0.0,0.00000,0.000000,0.000000,0.00000,0.000000,0.000000,0.943963


In [49]:
reports_df.index = reports_df.index.tz_localize('UTC')\
                      .tz_convert('Europe/London')\
                      .tz_localize(None)

In [50]:
reports_df['new_index'] = reports_df.index

In [51]:
reports_df['new_index'] = reports_df['new_index'].apply(lambda x: x.replace(second=5*(x.second//5)))
reports_df.set_index('new_index', inplace=True)

## Load volume bars and time bars

In [52]:
perpetual_bar_path='./../clean_data/bar_obj_perpetual_ts_{}{}.pickle'.format(RES, RES_UNIT)
resampled_bar_path='./../clean_data/bar_obj_volume{}_perpetual_ts.pickle'.format(VOLUME)

In [53]:
perpetual_ts = pd.read_pickle(perpetual_bar_path).df[[]]
resampled = pd.read_pickle(resampled_bar_path).df[['Volume_bucket_ID', 'bucket_start']]

## Build LUT

In [54]:
lut  = perpetual_ts.join(resampled.reset_index().set_index('bucket_start'))
lut['Volume_bucket_ID'] = lut['Volume_bucket_ID'].fillna(method='ffill')
lut['bucket_end'] = lut['bucket_end'].fillna(method='ffill')

In [56]:
reports_df = reports_df.join(lut)
reports_df['is_report'] = True
reports_df.to_pickle('../processed_data/features/nlp_reports/nlp_features_with_bucket_ts.pkl')

In [34]:
tmp = tmp.iloc[:, -2:]

In [35]:
tmp['dow'] = tmp.index.weekday
tmp['market_is_open'] = (tmp.index.time<=datetime.time(23,0,0)) & (tmp.index.time>=datetime.time(1,0,0)) & (tmp['dow']<5)

In [36]:
tmp['is_not_ok'] = tmp['bucket_end'].isnull()

In [37]:
tmp.groupby(['market_is_open'])['is_not_ok'].sum()

market_is_open
False     210.0
True     1064.0
Name: is_not_ok, dtype: float64

In [39]:
tmp

,bucket_end,Volume_bucket_ID,dow,market_is_open,is_not_ok
new_index,,,,,
2012-12-24 08:02:00,2012-12-24 08:05:15,38.0,0,True,False
2013-01-01 06:35:00,NaT,NaN,1,True,True
2013-01-04 18:42:00,2013-01-04 18:42:00,1049.0,4,True,False
2013-01-07 07:39:00,2013-01-07 07:43:20,28.0,0,True,False
2013-01-07 08:21:00,2013-01-07 08:21:30,58.0,0,True,False
2013-01-07 10:01:00,2013-01-07 10:01:00,195.0,0,True,False
2013-01-08 06:44:00,2013-01-08 06:49:45,26.0,1,True,False
2013-01-08 08:14:00,2013-01-08 08:14:20,51.0,1,True,False
2013-01-08 11:14:00,2013-01-08 11:14:10,276.0,1,True,False


In [38]:
tmp.query('is_not_ok & market_is_open')

,bucket_end,Volume_bucket_ID,dow,market_is_open,is_not_ok
new_index,,,,,
2013-01-01 06:35:00,NaT,NaN,1,True,True
2013-03-29 07:49:00,NaT,NaN,4,True,True
2015-04-03 15:41:00,NaT,NaN,4,True,True
2016-01-01 13:10:00,NaT,NaN,4,True,True
2017-01-02 02:39:00,NaT,NaN,0,True,True
2017-04-14 01:34:00,NaT,NaN,4,True,True
2017-04-14 07:34:00,NaT,NaN,4,True,True
2017-10-16 14:35:00,NaT,NaN,0,True,True
2017-10-17 15:47:05,NaT,NaN,1,True,True
